In [1]:
from stock_prediction.data_prep import Data_Prep
from stock_prediction.features_exo import exo_selection

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error as MAPE

<ipython-input-2-fac18f238935>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [ ]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)
 warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)

In [ ]:
# instantiate the Class 
vinci = Data_Prep('vinci', 10)

In [ ]:
# load data 
data_global = vinci.data_prep()

In [ ]:
# select fetarures 
data_return = vinci.select_features(data_global, Return = True, Log_Return=False, High_Low=False, High_Close=False, Low_Close=False,
                        Volume_Change=False, Period_Volum=False, Annual_Vol=False,
                        Period_Vol=False, Return_Index=False, Volum_Index=False, Relative_Return=False)

In [ ]:
# add exogenous features 
data_exo = exo_selection(data_return, sp500=True, gold=True, eurusd=True, nasdaq=True, crude=True, vix=True)

In [ ]:
# select first year 
data_exo = data_exo[-258:]

In [ ]:
# rebase the indices 
data_exo = vinci.Price_Rebase(data_exo)

In [ ]:
# create the y
y_vinci = data_exo['Return_DG.PA_R']
y_sp = data_exo['Return_S&P500_R']
y_usd = data_exo['Return_Usd_R']
y_nasdaq = data_exo['Return_Nasdaq_R']
y_crude = data_exo['Return_Crude_R']
y_vix = data_exo['Vix_No_Rebase']

In [ ]:
# assign the exogenous features 
y_endo = y_vinci
y_exo = y_vix

In [6]:


#empty lists to store results 
orders = []
mapeS = []
aics = []
list_y_pred = []
list_y_test = []
baseline = []
base_score = []
real_value = []

# our parameter 
order = (0,1,0)
max_train = 11
splits = len(y_endo) - max_train

# we create all the index for slices covering all the TS
folds = TimeSeriesSplit(n_splits=splits, max_train_size=max_train, test_size=1)

for (train_idx, test_idx) in folds.split(y_endo) :
   
    # we retrieve the real data in y 
    #corresponding of the indexes splited 
    y_train = y_endo[train_idx]
    y_exo_train = y_exo[train_idx]
    y_test = y_endo[test_idx]
    y_exo_test = y_exo[test_idx]
    y_true = y_endo[test_idx]
    base = y_train.iloc[-1]

 
    # fit our model on y_train of the split n°X
    model = ARIMA(endog=y_train, exog=y_exo_train, order=order).fit()
    
    # we find our y_pred for this slice on that part of the TS
    y_pred = model.forecast(steps=len(y_test), exog=y_exo_test)[0]
    # we store y_pred
    list_y_pred.append(y_pred[0])
    # we store the value for basescore
    baseline.append(base)
    # and the rel value to compare 
    real_value.append(y_true.values[0])


# once we have all values we can compute MAPE and basescore   
base_score = MAPE(np.array([base]), y_true)
mape_metric = MAPE(y_pred, y_true)


NameError: name 'y_endo' is not defined

In [ ]:
base_score

In [ ]:
mape_metric

In [ ]:
# we store this results on the table on Notion 